In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import os
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
path_archivos = "./Data/"
os.listdir(path_archivos)

['InversionComp.csv',
 'Sales.csv',
 'SpecialSale.csv',
 'InversionMedia.csv',
 'Sector.csv']

In [5]:
# Eccomerce CF
InversionMedia = pd.read_csv(path_archivos + "InversionMedia.csv")
Sales = pd.read_csv(path_archivos + "Sales.csv")
SpecialSale = pd.read_csv(path_archivos + "SpecialSale.csv")
# Total Sectores y competidores
InversionComp = pd.read_csv(path_archivos + "InversionComp.csv")
Sector = pd.read_csv(path_archivos + "Sector.csv")

## EDA

In [14]:
data_CF = pd.merge(InversionMedia, Sales, on="Date", how="left")
data_CF = pd.merge(data_CF, SpecialSale, on="Date", how="left").rename(columns={"Pay TV": "PayTV", "Open TV": "OpenTV", "Sales Name": "SpecialSale"})

El Dataset cuenta con 12 variables/features:
- 1: tiempo - Date 
- 2: inversion - Print 
- 3: inversion - Other
- 4: inversion - Email
- 5: inversion - Radio      
- 6: inversion - Facebook    
- 7: inversion - Google      
- 8: inversion - PayTV       
- 9: inversion - OpenTV      
- 10: Ventas de la ecommers - Sales       
- 11: Score de Recomendacion de la pagina - NPS         
- 12: Dias de Ventas Especiales - SpecialSale (contiene valores nulos correspondientes a los dias que no son ventas especiales)

In [15]:
data_CF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105 entries, 0 to 104
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Date         105 non-null    object
 1   Print        105 non-null    int64 
 2   Other        105 non-null    int64 
 3   Email        105 non-null    int64 
 4   Radio        105 non-null    int64 
 5   Facebook     105 non-null    int64 
 6   Google       105 non-null    int64 
 7   PayTV        105 non-null    int64 
 8   OpenTV       105 non-null    int64 
 9   Sales        105 non-null    int64 
 10  NPS          105 non-null    int64 
 11  SpecialSale  37 non-null     object
dtypes: int64(10), object(2)
memory usage: 10.7+ KB


In [16]:
data_CF['SpecialSale'] = data_CF['SpecialSale'].fillna("None")
data_CF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105 entries, 0 to 104
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Date         105 non-null    object
 1   Print        105 non-null    int64 
 2   Other        105 non-null    int64 
 3   Email        105 non-null    int64 
 4   Radio        105 non-null    int64 
 5   Facebook     105 non-null    int64 
 6   Google       105 non-null    int64 
 7   PayTV        105 non-null    int64 
 8   OpenTV       105 non-null    int64 
 9   Sales        105 non-null    int64 
 10  NPS          105 non-null    int64 
 11  SpecialSale  105 non-null    object
dtypes: int64(10), object(2)
memory usage: 10.7+ KB


In [17]:
data_CF.describe().T

,count,mean,std,min,25%,50%,75%,max
Print,105.000,1733.333,1744.589,0.000,0.000,1000.000,3000.000,6000.000
Other,105.000,1209.524,1149.375,0.000,0.000,1000.000,2000.000,5000.000
Email,105.000,3504.762,2197.568,0.000,2000.000,3000.000,5000.000,7000.000
Radio,105.000,485.714,556.678,0.000,0.000,0.000,1000.000,3000.000
Facebook,105.000,8971.429,4923.369,1000.000,5000.000,9000.000,12000.000,17000.000
Google,105.000,10895.238,5448.930,1000.000,6000.000,12000.000,15000.000,22000.000
PayTV,105.000,10019.048,11902.635,0.000,1000.000,5000.000,15000.000,55000.000
OpenTV,105.000,19409.524,21932.017,0.000,0.000,10000.000,35000.000,89000.000
Sales,105.000,73759.400,48456.297,3680.000,44928.000,61207.000,92684.000,284575.000
NPS,105.000,60.295,16.230,26.000,49.000,63.000,72.000,95.000


In [49]:
varible = "Print"
px.histogram(data_CF, x=varible,
            marginal="box",
            title="Histograma de " + varible)